In [1]:
# ---- IMPORTING ----
import math,re
#import xlsxwriter
import csv
import pymatgen
import pandas as pd 
import numpy as np
import matplotlib.ticker as ticker
import sys

from pymatgen.core import Element
from pymatgen.ext.matproj import MPRester
from pymatgen.core import Structure
from pymatgen.analysis.adsorption import *
from pymatgen.analysis.structure_analyzer import VoronoiConnectivity
from pymatgen.analysis.chemenv.coordination_environments.voronoi import DetailedVoronoiContainer
from pymatgen.analysis.local_env import  VoronoiNN

In [6]:
# %% ---- CLASS_START ----
class hope2:
    # ---- CONSTRUCTOR ----
    def __init__(self):       
        He    = [1,'s',2]
        Ne    = [He,[2,"s",2],[2,'p',6]]
        Ar    = [Ne,[ 3,'s',2],[3,'p',6]]
        Kr    = [Ar,[4,'s',2],[3,'p',6],[3,'d',10]] 
        ''' 1. Yogi '''
        Xe    = [Kr,[5,'s',2],[4,'d',10],[5,'p',6]]
        Rn    = [Xe,[6,'s',2],[4,'f',14],[5,'d',10],[6,'p',6]]

        # Elements symbol
        self.elementsAb = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne", "Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K",
                        "Ca", "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn", "Ga", "Ge", "As", "Se", "Br", "Kr", "Rb",
                        "Sr", "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd", "In", "Sn", "Sb", "Te", "I", "Xe", "Cs",
                        "Ba", "La", "Ce", "Pr", "Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu", "Hf",
                        "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", "Po", "At", "Rn", "Fr", "Ra", "Ac", "Th",
                        "Pa", "U", "Np", "Pu", "Am", "Cm", "Bk", "Cf", "Es", "Fm", "Md", "No", "Lr", "Rf", "Db", "Sg", "Bh", "Hs",
                        "Mt", "Ds", "Rg", "Cn", "Nh", "Fl", "Mc", "Lv", "Ts", "Og"]
        # Elements Name
        self.elementsFull = ["Hydrogen", "Helium", "Lithium", "Beryllium", "Boron", "Carbon", "Nitrogen", "Oxygen", "Fluorine", "Neon",
                          "Sodium", "Magnesium", "Aluminium", "Silicon", "Phosphorous", "Sulfur", "Chloride", "Argon", "Potassium",
                          "Calcium", "Scandium", "Titanium", "Vanadium", "Chromium", "Manganese", "Iron", "Cobalt", "Nickle",
                          "Copper", "Zinc", "Gallium", "Germanium", "Arsenic", "Selenium", "Bromine", "Krypton", "Rubidium", "Strontium",
                          "Yttrium", "Zirconium", "Niobium", "Molybdenum", "Technetium", "Ruthenium", "Rhodium", "Palladium", "Silver",
                          "Cadmium", "Indium", "Tin", "Antimony", "Tellurium", "Iodine", "Xenon", "Cesium", "Barium", "Lanthanum",
                          "Cerium", "Praseodymium", "Neodymium", "Promethium", "Samarium", "Europium", "Gadolinium", "Terbium",
                          "Dysprosium", "Holmium", "Erbium", "Thulium", "Ytterbium", "Lutetium", "Hafnium", "Tantalum", "Tungsten",
                          "Rhenium", "Osmium", "Iridium", "Platinum", "Gold", "Mercury", "Thallium", "Lead", "Bismuth", "Polonium",
                          "Astatine", "Radon", "Francium", "Radium", "Actinium", "Thorium", "Protactinium", "Uranium", "Neptunium",
                          "Plutonium", "Americium", "Curium", "Berkelium", "Californium", "Einsteinium", "Fermium", "Mendelevium",
                          "Nobelium", "Lawrencium", "Rutherfordium", "Dubnium", "Seaborgium", "Bohrium", "Hassium", "Meitnerium"]

 # ---- DISTANCE ---- function to find distance between site index and nearest neighbour ----
    def distance(self,a,b):
        result = math.sqrt(pow(a[0]-b[0],2)+pow(a[1]-b[1],2)+pow(a[2]-b[2],2))        
        return result
        
    # ---- ATOMIC_NUMBER ---- function to fetch atomic_number ----
    def atom_no(self,a):
        n = self.elementsAb.index(a)+1 
        ''' 2. Yogi '''
        return n
     
    # ---- ELECTRONIC FUNCTION ---- electronic configuration of al compounds in form of noble gas----
    def e_config(self,a):
        n = Element(a)
        configuration = n.full_electronic_structure
        atomic = n.Z
        if atomic > 1:
          if atomic >= self.atom_no('He'):
              if atomic >= self.atom_no('Ne'):
                  if atomic >= self.atom_no('Ar'):
                      if atomic >= self.atom_no('Kr'):
                          if atomic >= self.atom_no('Xe'):
                              if atomic >= self.atom_no('Rn'):
                                  configuration[14] = "Rn"
                                  configuration = configuration[14:]
                                  return(configuration)
                              else:
                                  configuration[10] = "Xe"
                                  configuration = configuration[10:]
                                  return(configuration)
                          else:
                              configuration[7] = "Kr"
                              configuration = configuration[7:]
                              return(configuration)
                      else:
                          configuration[4] = "Ar"
                          configuration = configuration[4:]
                          return(configuration)
                  else:
                      configuration[2] = "Ne"
                      configuration = configuration[2:]
                      return(configuration)
              else:
                  configuration[0] = "He"
                  return(configuration)    #n.full_electronic_structure - returns electronic configuration & n.Z returns atomic number
        else:
            return(['H', (1, 's', 1)])

    def ohv(self, a): # a= ELement Symbol and nobal gas are all zeros, tested till Lr
      element = self.e_config(a)[1:]       
      element = [l[1:] for l in element]
      #print(element)      
      #arr = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
      arr = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
      #print(arr[30])
      suuu='s'
      for z in range(len(element)):
        if suuu in element[z][0]:
          #print(element[z][0])
          yy=element[z][1]
          for ab in range(yy):
            if ab<1:
                arr[ab]=arr[ab]+1
            else:
                arr[ab-1]=arr[ab-1]-1
            
      suuu='d'
      for z in range(len(element)):
        if suuu in element[z][0]:
          #print(element[z][0])
          yy=element[z][1]
          for ab in range(yy):
            if ab<5:
                arr[ab+4]=arr[ab+4]+1
            else:
                arr[ab-1]=arr[ab-1]-1
          #print(yy)
        
      suuu='p'
      for z in range(len(element)):
        if suuu in element[z][0]:
          #print(element[z][0])
          yy=element[z][1]
          for ab in range(yy):
            if ab<3:
                arr[ab+1]=1  
            else:
                arr[ab-2]=arr[ab-2]-1
            
      suuu='f'
      for z in range(len(element)):
        if suuu in element[z][0]:
          #print(element[z][0])
          yy=element[z][1]
          for ab in range(yy):
            if ab<7:
                arr[ab+9]=1
            else:
                arr[ab+2]=arr[ab+2]-1
      #print(arr)    
      arr=np.array([arr])       
      return(arr)

    # atom name of site 
    def site_a(self,ind): # pass the imdex
      sit_ato = struct[ind].species
      site     = ''.join([i for i in str(sit_ato) if not i.isdigit()])
      return(site)

# ---- CLASS_END ----

In [11]:
# CLASS objects
p = hope2()
vv = VoronoiNN(tol=0.1)
refer = [('s',1),('s',2),('p',1),('p',2),('p',3),('p',4),('p',5),('p',6),('d',1),('d',2),('d',3),('d',4),('d',5),('d',6),('d',7),('d',8),('d',9),('d',10),('f',1),('f',2),('f',3),('f',4),('f',5),('f',6),('f',7),('f',8),('f',9),('f',10),('f',11),('f',12),('f',13),('f',14)]

In [13]:
m = MPRester("x8fEcBqExHiMPxUn6")
data_s = m.query(criteria={"elements": {"$all": ["Mn"]}}, properties=["task_id","full_formula","magnetism.magmoms","total_magnetization","structure","spacegroup.crystal_system","nsites","spacegroup.number","magnetism.ordering","formation_energy_per_atom"])
df = pd.DataFrame(data_s)
print(df.shape)

(11707, 10)


In [15]:
#df.to_csv('magmoms.csv')

In [5]:
df["MPerAtom"] = "NaN"
df['count'] = 'NaN'
df['total_magnetization'] = df['total_magnetization'].astype(float)
df['nsites'] = df['nsites'].astype(float)
df['MPerAtom'] = df['total_magnetization'] / df['nsites'] 

In [6]:
''' Function and loop '''
def ofm(struct,nsites):
  # Count of elements in a compound
  ###count = len(struct)
  count = nsites
  # nearest neighbor class
  nn = vv.get_all_nn_info(struct)
  #print(nn)
  # Orbital Field Matrix
  sum = np.zeros((16, 16))
  
  ''' index loop'''
  for i in range(int(count)):  ## count = len(nn)
    site = nn[i]
    #print(site)
    s_atom = p.site_a(i) # name of site atom
    #print(s_atom)
    s_ohv = p.ohv(s_atom) # one hot vector of site
    #print(s_ohv)
    #ang = []

    ''' Neighbor loop'''
    for j in (nn[i]):
      ## neighbor_detail = j
      
      ''' 4. Yogi - multiplied by 2'''
      dist_nn = 2 * j['poly_info']['face_dist'] # distance of neighbor mult 2
      
      ''' 5. Yogi '''
      #weight_n = j['weight'] #weight normalized solid angle  
      #angle_nn = j['poly_info']['solid_angle'] # angle of neighbor
      n_ind = j['site_index'] # neighbors index
      n_atom = p.site_a(n_ind) # name of neighbor atom
      n_ohv = p.ohv(n_atom) # one hot vector of neighbor done 
      s_ohv = np.asarray(s_ohv)
      n_ohv = np.asarray(n_ohv)#done
      #print(s_ohv)
      #ang.append(angle_nn)
      # Neighboring matrix is multiplied first

      final = np.dot(s_ohv.T,n_ohv) * (1/(dist_nn*dist_nn)) #neighbour is second    
      #print(final)
      sum = np.add(sum, final)
       
  return sum.flatten()

In [7]:
#c = np.vstack((ofm(struct)))
exc = []
struct = df.iloc[0]['structure']
sites = df.iloc[0]['nsites']
matr = ofm(struct,sites)
# for rest of them
a = np.zeros((1,256))
for i in range(1,df.shape[0]):
#for i in range(1,100):
  if i%100==0:
    print(i)
  try:
    struct = df.iloc[i]['structure']
    sites = df.iloc[i]['nsites']
    matr = np.vstack((matr,ofm(struct,sites)))
  except:
    print(sys.exc_info()[0])
    matr = np.vstack((matr,a[0]))
    exc.append(df.iloc[i]['task_id'])
df2 = pd.DataFrame(matr)
print(exc)

# merge fetched data dataframe wth that of ofm
# ofm - 1024 length 
# new dataframe - all 1024 values in new column


100
200
<class 'ValueError'>
<class 'ValueError'>
300
400
500
600
<class 'ValueError'>
700
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
800
900
1000
<class 'ValueError'>
1100
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
1200
1300
1400
1500
<class 'ValueError'>
1600
<class 'ValueError'>
1700
<class 'ValueError'>
1800
1900
2000
<class 'ValueError'>
<class 'ValueError'>
2100
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
2200
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
2300
2400
2500
<class 'ValueError'>
2600
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
2700
2800
2900
3000
<class 'ValueError'>
3100
<class 'ValueError'>
3200
3300
<class 'ValueError'>
3400
3500
<class 'ValueError'>
3600
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'V

In [8]:
len(exc)

94

In [9]:
df3 = df.join(df2)
df2 = df3.copy()
dp = []
# Find mpid of elements whose stucture properties were not fetched
for i in exc:
    dp.append(df2.loc[df2.task_id == i ].index[0])
# Remove all those
df2.drop(index=dp,inplace=True)
# Null columns
[col for col in df2.columns if df2[col].isnull().any()]
#df2.shape
df2.to_csv('Hunds-matrix.csv')